In [6]:
import numpy as np
from glob import glob
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader

# Set PIL to be tolerant of image files that are truncated.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torchvision.models as models

# define VGG16 model
VGG16 = models.vgg16(pretrained=True)


# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    VGG16 = VGG16.cuda()


image = Image.open('dogImages/train/001.Affenpinscher/Affenpinscher_00001.jpg').convert('RGB')
    
    
    
in_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),
                             (0.229, 0.224, 0.225))
    ])
    
image = in_transform(image)[:3, :, :].unsqueeze(0)
    
dataLoader = DataLoader(image, batch_size=1, num_workers=1, shuffle=True)
    
for i in iter(image):
    image = i.cuda()
    
prediction = VGG16(image)
    
prediction = prediction.data.numpy().argmax()  # Our prediction will be the index of the class label with the largest value.

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 3, 3], but got 3-dimensional input of size [3, 300, 400] instead